# 10_정수 인코딩(Integer Encoding)

컴퓨터는 텍스트보다는 숫자를 더 잘 처리한다. 그 때문에 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있다. 그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있다. 
<br>

예를 들어 갖고 있는 텍스트가 단어 5,000개로 구성되어 있다면, 5,000개의 단어들 각각에 1번부터 5,000번까지 맵핑되는 고유한 정수, 다른 표현으로 말하자면 인덱스를 부여하는 것이다. 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 숫자를 부여한다고 생각하면 된다. 이러한 인덱스를 부여하는 방법은 여러 가지가 있을 수 있다. 랜덤으로 부여하기도 하지만, 보통은 전처리 또는 빈도수가 높은 단어들만 사용하기 위해 단어 출현 빈도수를 기준으로 정렬한 후에 인덱스를 준다.

## 1. 정수 인코딩(Integer Encoding) 
왜 이러한 작업이 필요한지에 대해서는 뒷부분의 [원 핫 인코딩](), [워드 임베딩]() 챕터 등에서 알아보기로 하고 여기서는 어떤 과정으로 단어에 정수 인덱스를 부여하는지에 대해서만 정리한다. 
<br>

단어에 정수(인덱스)를 부여하는 방법 중에는 우선 단어 출현 빈도 순으로 정렬한 단어 집합(vocabulary)를 만들고, 이 빈도수가 높은 순서대로 1, 2, 3....과 같이 정수 인덱스를 부여하는 방법이 있다. 아래에서는 이해를 돕기 위해 단어의 빈도수가 적당히 분포되도록 구축한 텍스트 데이터를 가지고 실습한다.  

### 1) dictionary 사용하기

In [1]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

### 여러 문장이 함께 있는 텍스트 데이터 문장 토큰화

In [3]:
# 문장 토큰화 
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


기존 텍스트 데이터가 문장 단위로 토큰화되는 것을 확인할 수 있다. 이제 정제 작업을 병행하여, 단어 토큰화를 수행한다. 

In [4]:
# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence = word_tokenize(i)
    result = []
    
    
    for word in sentence:
        word = word.lower() # 모든 단어를 소문자로 변경, 단어의 개수를 줄임
        
        if word not in stop_words: # 단어 토큰화된 결과에 대해 불용어를 제거함
            if len(word) > 2: # 단어 길이가 2 이하인 경우 추가 제거함
                result.append(word)
                
                if word not in vocab:
                    vocab[word] = 0 # key 값과 value값 채워넣기
                vocab[word] += 1
    sentences.append(result)
print(sentences)         

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


텍스트를 숫자로 바꾸는 단계라는 것은 본격적인 자연어 처리 작업에 들어간다는 의미이다. 단어가 텍스트일 때만 할 수 있는 최대한의 전처리를 끝내놓아야만 한다. 위의 코드를 보면, 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트되는 일이 없도록, 모든 단어를 소문자로 바꿨다. 또한 자연어 처리에서 크게 의미가 없는 불용어와 길이가 짧은 단어를 제거했다. 
<br>

그 결과 현재 vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록되어 있다. vocab을 출력해보자.

In [5]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


단어가 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있다. vocab에 단어를 입력하면 빈도수를 리턴한다. 

In [6]:
print(vocab["person"]) # 'person'이라는 단어의 빈도수 출력

3


빈도수가 높은 순서대로 정렬해본다.

In [7]:
vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


이제 인덱스를 부여합니다. 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여합니다.

In [8]:
word_to_index = {}
i=0
for (word, frequency) in vocab_sorted:
    if frequency > 1: # 빈도수가 적은 단어 제외
        i=i+1
        word_to_index[word] = i
        
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


가장 빈도수가 높은 단어가 1의 인덱스를 가진다. 이러한 작업을 수행하는 동시에 각 단어의 빈도수를 알 경우에만 할 수 있는, 빈도수가 적은 단어를 제외하는 작업을 한다. 등장 빈도가 적은 단어는 자연어 처리에서 큰 의미가 없기 때문이다. 
<br>

자연어 처리를 하다 보면, 텍스트 데이터에 있는 단어를 몯 사용하기보다 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많다. 위 단어들은 빈도수가 높은 순으로 낮은 정수가 부여되므로 빈도수가 높은 상위 n개의 단어들만 사용하고 싶다면 vocab에서 정수 인덱스가 1에서 n까지인 단어들만 사용하면 된다. 여기서는 상위 5개 단어만 사용한다.

In [9]:
vocab_size=5 # 상위 5개 단어
words_frequency=[w for w, c in word_to_index.items() if c >= vocab_size +1] # 인덱스가 5 초과인 단어를 제거
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보 삭제 
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


이제 word_to_index에는 빈도수가 높은 상위 5개의 단어만 저장되었다. 이제 word_to_index를 사용해, 단어 토큰화만 된 상태로 저장된 sentences의 각 단어를 정수로 바꾸는 작업을 진행한다. 
<br>

예를 들어 sentences의 첫 번째 문장은 ['barber', 'person']이었는데, 이 문장에 대해서는 [1,5]로 인코딩한다. 그런데 두 번째 문장인 ['barber', 'good', 'person']에는 word_to_index에 존재하지 않는 단어 'good'이라는 단어가 있다. 
<br>

이처럼 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 'OOV'라고 한다. word_to_index에 'OOV'를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩한다. 

In [10]:
word_to_index['OOV'] = len(word_to_index) + 1

이제 word_to_index를 사용해 sentences의 모든 단어들을 인덱스와 맵핑시켜 인코딩하겠습니다. 

In [11]:
encoded = []
for s in sentences:
    temp=[]
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


지금까지 파이썬의 dictionary 자료형으로 정수 인코딩을 진행해봤다. 이보다 더 쉬운 인코딩 방법으로 Counter, FreqDist, enumerate 또는 케라스 토크나이저를 사용할 것을 권장한다.

### 2) Counter 사용하기 

In [12]:
from collections import Counter

In [13]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


현재 sentences는 단어 토큰화가 된 결과가 저장되어 있다. 단어 집합(vocabulary)를 만들기 위해 sentences에서 문장의 경계인 [,]를 제거하고 단어들을 하나의 리스트로 만든다. 

In [16]:
words = sum(sentences, [])
# 위의 작업은 words = np.hstack(sentences)로도 수행할 수 있다. 
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


이를 Counter()의 입력으로 사용하면 중복을 제거하고 단어의 빈도수를 기록할 수 있다. 

In [17]:
vocab = Counter(words) # 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산할 수 있음
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


단어가 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있다. vocab에 단어를 입력하면 빈도수를 리턴한다. 

In [18]:
print(vocab['person']) # 'person'이라는 단어의 빈도수 출력

3


person이라는 단어는 총 3번 등장했다. most_common(n)은 상위 빈도수를 가진 단어들 중 빈도수 순위 1위~n위까지의 단어들을 리턴한다. 이를 사용해 등장 빈도수가 높은 단어들을 원하는 개수만큼 얻을 수 있다. 등장 빈도수 상위 5개의 단어만 단어 집합으로 저장해 보자. 

In [19]:
vocal_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수 1~5위까지의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

이제 높은 빈도수를 가진 단어순으로 1부터 인덱스를 부여한다. 

In [20]:
word_to_index = {} # dictionary 생성
i = 0
for (word, frequency) in vocab : # 빈도순으로 내림차순 정렬된 vocab의 item들을 하나씩 꺼내옴
    i = i+1 # 정수 인덱스가 1씩 증가
    word_to_index[word] = i # 인덱스 값 지정
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### 3) NLTK의 FreqDist 사용하기
NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원한다. 위에서 사용한 Counter()와 같은 방법으로 사용할 수 있다. 

In [21]:
from nltk import FreqDist
import numpy as np

In [22]:
# np.hstack으로 문장 구분([])을 제거하여 입력으로 사용 ex)['barber', 'person'. 'barber', 'good'....]
vocab = FreqDist(np.hstack(sentences))

단어가 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있다.vocab에 단어를 입력하면 빈도수를 리턴한다. 

In [23]:
print(vocab['person']) # 'person'의 빈도수 출력

3


'person'이란 단어가 총 3번 등장했다. most_common(n)은 1위~n위까지의 상위 빈도수 단어들을 리턴한다. 등장 빈도수 5위까지의 단어만 단어 집합으로 저장해 본다. 

In [24]:
vocal_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

앞서 Counter()를 사용했을 때와 결과가 같다. 이전 실습들과 마찬가지로 높은 빈도수를 가진 단어 순으로 1부터 정수 인덱스를 부여한다. 하지만 이번에는 enumerate()를 사용해 좀 더 짧은 코드로 인덱스를 부여하겠다. 

In [25]:
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### 4) enumerate 이해하기
enumerate()는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴한다는 특징이 있다. 간단한 예제를 통해 enumerate()를 이해해본다. 

In [1]:
test = ["a", "b", "c", "d", "e"]
for index, value in enumerate(test): # 입력의 순서대로 0부터 인덱스를 부여
    print("value : {}, index : {}".format(value, index))

value : a, index : 0
value : b, index : 1
value : c, index : 2
value : d, index : 3
value : e, index : 4


## 2. 케라스(Keras)의 텍스트 전처리
케라스(Keras)는 기본적인 전처리를 위한 도구들을 제공한다. 때로는 정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저를 사용하기도 하는데, 사용 방법과 그 특징에 대해서 이해해보자. 

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

단어 토큰화가 완료된 앞의 데이터를 그대로 사용한다. 

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts() 안에 코퍼스를 입력하면 빈도수 기준으로 단어 집합 생성

fit_on_texts는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여하는데, 앞서 설명한 정수 인코딩 작업이 이루어진다고 보면 된다. 각 단어에 인덱스가 어떻게 부여되었는지를 보려면, word_index를 사용한다. 

In [7]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


앞서 빈도수가 가장 높은 단어 n개만을 사용하기 위해서 most_common()을 사용했다. 케라스 토크나이저에서는 tokenizer = Tokenizer(num_words=n)와 같은 방법으로 빈도수가 높은 상위 n개의 단어들만 사용하겠다고 지정할 수 있다. 여기서는 1~5위까지의 단어만 사용해보겠다. 

In [8]:
vocab_size = 5
tokenizer = Tokenizer(num_words=vocab_size + 1) # 상위 5개의 단어만 사용
tokenizer.fit_on_texts(sentences)

num_words에서 +1를 더해서 값을 넣어주는 이유는 num_words는 숫자를 0부터 카운트하기 때문이다. 만약 5를 넣으면 0~4번 단어 보존을 의미하게 되므로 뒤의 실습에서 1번 단어부터 4번 단어만 남게 된다. 그렇기 때문에 1~5번 단어까지 사용하고 싶다면 num_words에 숫자 5를 넣는 게 아니라 5+1인 값을 넣어줘야 한다. 
<br>
실질적으로 숫자 0에 지정된 단어가 존재하지 않는데도 케라스 토크나이저가 숫자 0까지 단어 집합의 크기로 산정하는 이유는 자연어 처리에서 패딩(padding)이라는 작업 때문이다. 이에 대해서는 후에 다루게 되므로 케라스 토크나이저 사용 시 숫자 0도 단어 집합의 크기로 고려한다고만 이해해두자.
<br>
다시 word_index를 확인해본다. 

In [9]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


상위 5개 단어만 사용하겠다고 했는데 여전히 13개의 단어가 모두 출력된다. word_counts를 확인해본다. 

In [10]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


word_counts에서도 마찬가지로 13개의 단어가 모두 출력된다. 실제 적용은 texts_to_sequences를 사용할 때 적용된다. 

In [11]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


코퍼스에 대해서 각 단어를 정해진 인덱스로 변환하는데, 상위 5개 단어만을 사용하겠다고 지정했으므로 1번부너 5번 단어까지만 보존되고 나머지 단어들은 제거된 것을 볼 수 있다. 필요한 때가 있을지는 모르겠으나, 만약 word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면 아래의 코드도 방법이 될 수 있다. 

In [12]:
tokenizer = Tokenizer() # num_words를 지정하지 않음
tokenizer.fit_on_texts(sentences)

In [13]:
vocab_size = 5
words_frequency = [w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del tokenizer.word_index[w] # 해당 단어에 대한 인덱스 정보 삭제
    del tokenizer.word_counts[w] # 해당 단어에 대한 카운트 정보 삭제
    
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있다. 단어 집합에 없는 단어들을 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용한다. 

In [15]:
vocab_size = 5
tokenizer = Tokenizer(num_words= vocab_size + 2, oov_token = 'OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려하여 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)

만약 oov_token을 사용하기로 했다면 케라스 토크나이저는 기본적으로 'OOV'의 인덱스를 1로 한다. 

In [16]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


이제 코퍼스에 대해서 정수 인코딩을 진행한다. 

In [17]:
print(tokenizer.texts_to_sequences(sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


빈도수 상위 5개의 단어는 2~6까지의 인덱스를 가졌고, 그 외 단어 집합에 없는 'good'과 같은 단어는 전부 'OOV'의 인덱스인 1로 인코딩되었다. 